<h1 align='center'>MSDE 692</h1>

[Global Variables](#Global-Variables)

[Function Definitions](#Function-Definitions)

[Data Cleaning](#Data-Cleaning)

In [342]:
import os
import pandas as pd
import neo4j
import numpy as np
import csv
import shutil
import urllib
import codecs

from scrapy.crawler import CrawlerProcess
from pprint import pprint
from deepdiff import DeepDiff
from functools import reduce
from datetime import datetime as dt
from csv import DictReader, writer

In [5]:
# https://www.youtube.com/watch?v=5Is-QdbKmEI

## Global Variables

In [6]:
OMIT_LIST = ['Wikipedia:', 'Â', 'Ã']

## Function Definitions

In [266]:
def csv_reader(csv_file):
    final_list = []

    with open(csv_file, 'r', encoding='unicode_escape') as f:
        input_list = csv.reader(f)

        for row in input_list:
            final_list.append(row[0])
            
    return final_list

In [204]:
def people_pruner(my_file):
    my_list = []

    with open(my_file, 'r') as f:
        reader = DictReader(f, fieldnames='person')

        for row in reader:
            if [row['p']] not in my_list:
                my_list.append([row['p']])
            else:
                print(f"Duplicate person: {row['p']}")
    
    return my_list

In [318]:
def csv_writer(my_file, my_mode, my_list):

    with open(my_file, my_mode, newline='', encoding="utf-8") as outfile:
        csv_writer = writer(outfile)

        for row in my_list:
            if not (":" in row or row == 'wiki' or row.startswith(tuple(OMIT_LIST)) or "Ã" in row):
                csv_writer.writerow([row])

In [270]:
def csv_dict_writer(my_file, my_list, my_mode):
    with open(my_file, my_mode, newline='') as f:
        writer = csv.DictWriter(f, fieldnames=['wiki'])
        
        for row in my_list:
            if ":" not in row and row != 'wiki' and not row.startswith(tuple(OMIT_LIST)):
                writer.writerow({'wiki': row})

In [271]:
def add_new_wikis(adding_file, gaining_file, base_file):
    """
    Adds newly discovered wiki pages to an accumulative wiki pages file before deleting the previous base file
    used to extract the new wikis. The previous base file is then deleted and the adding_file is renamed to base_file
    for the next grabwikis spider run.
    """
    base_list_tmp = csv_reader(base_file)
    adding_list = csv_reader(adding_file)
    gaining_list = csv_reader(gaining_file)
    base_set = set(base_list_tmp)
    adding_set = set(adding_list)
    gaining_set = set(gaining_list)
    
    new_base_wikis = list(set.difference(base_set, gaining_set))
    new_wikis = list(set.difference(adding_set, gaining_set))
    
    if len(new_wikis) > 0:
        print(f"Adding {len(new_wikis)} new wiki pages to {gaining_file}")
        csv_writer(gaining_file, 'a', new_wikis)
    if len(new_base_wikis) > 0:
        print(f"Adding {len(new_base_wikis)} new base wiki pages to {gaining_file}")
        csv_writer(gaining_file, 'a', new_base_wikis)
#         csv_dict_writer(base_file, new_wikis, 'w')
#         os.remove(adding_file)
    os.remove(base_file)
#     shutil.copy(adding_file, base_file)
    os.remove(adding_file)
    csv_writer(base_file, 'w', new_wikis)

In [383]:
def clean_df(my_df, out_file):
    all_people_df = my_df.drop_duplicates()
    all_people_df = all_people_df[all_people_df.wiki.str.contains(":")==False]
#     all_people_df['wiki'] = all_people_df['wiki'].apply(lambda x: urllib.parse.unquote(x))
    all_people_df = all_people_df[~all_people_df['wiki'].str[:].str.contains("Ã")]    
    all_people_df = all_people_df[~all_people_df['wiki'].str[0].str.isdigit()]    
    all_people_df.sort_values('wiki', inplace=True)
    all_people_df.set_index('wiki', drop=True, inplace=True)
    all_people_df.to_csv('wikigrabber/all_people.csv', encoding='utf-8')    

## Data Cleaning

### Create Difference CSV File for wikigrabber Spider Runs

In [360]:
add_new_wikis('wikigrabber/people.csv', 'wikigrabber/all_people.csv', 'wikigrabber/people_base.csv')

Adding 920 new wiki pages to wikigrabber/all_people.csv
Adding 430 new base wiki pages to wikigrabber/all_people.csv


In [385]:
df = pd.read_csv('wikigrabber/all_people.csv', encoding='unicode_escape')
df

,wiki
0,(Barbara)_Hazel_Guggenheim_King-Farlow_McKinley
1,A.A._Ames
2,A.P.J._Abdul_Kalam
3,A.T._Smith
4,A.V._Balakrishnan
...,...
68095,Ã Â½eljka_AntunoviÃÂ
68096,Ã Â½eljko_Reiner
68097,Ã Â½eljko_Ã Â turanoviÃÂ
68098,Ã Â½ivorad_KovaÃÂeviÃÂ


In [386]:
clean_df(df, 'wikigrabber/all_people.csv')

### Text Manipulation

In [120]:
df = pd.read_csv('wikigrabber/people.csv', encoding='unicode_escape')

In [123]:
df

,wiki
0,Franz_S._Exner
1,Karl_Herzfeld
2,Erwin_SchrÃ¶dinger
3,Adil_OsmanoviÄ
4,Igor_RadojiÄiÄ
...,...
2680,Ejup_GaniÄ
2681,KreÅ¡imir_Zubak
2682,Dario_KordiÄ
2683,Ejup_GaniÄ


In [24]:
df['name'] = df['name'].apply(lambda x: x.strip())
df['name'] = df['name'].apply(lambda x: x.replace(' ', '_'))
df

,name
0,Adewale_Adeyemo
1,Afsin_Yurdakul
2,Albert_Bourla
3,Alex_Karp
4,Ana_Pinho
...,...
398,Mark_Tucker
399,Jessica_Uhl
400,Ulrik_Vestergaard_Knudsen
401,Darren_Walker


In [276]:
df.to_csv('wikicrawler/all_people.csv')

In [278]:
all_people_df = df.drop_duplicates()
all_people_df = all_people_df[all_people_df.wiki.str.contains(":")==False]
all_people_df.sort_values('wiki', inplace=True)
all_people_df.set_index('wiki', drop=True, inplace=True)
all_people_df.to_csv('wikigrabber/all_people.csv')
all_people_df

""
wiki
(Barbara)_Hazel_Guggenheim_King-Farlow_McKinley
A.A._Ames
A.P.J._Abdul_Kalam
A.T._Smith
A.V._Balakrishnan
...
ÃÂÃÂtienne_Hirsch
ÃÂÃÂ¯ÃÂÃÂ»ÃÂÃÂ¿Edward_VIII
ÃÂ ÃÂ½ivko_RadiÃÂ ÃÂ¡iÃÂÃÂ


In [343]:
url_df = pd.read_csv('wikigrabber/all_people.csv', encoding='unicode_escape')
url_df

,wiki
0,%C3%81kos_Birtalan
1,%C3%81lvaro_Nadal
2,%C3%81ngel_Acebes
3,%C3%81ngeles_Amador
4,%C3%85sa_Lindhagen
...,...
68503,Zweli_Mkhize
68504,Zygmunt_Janiszewski
68505,Zygmunt_Zalcwasser
68506,Zyon_Braun


In [344]:
url_df['wiki'] = url_df['wiki'].apply(lambda x: urllib.parse.unquote(x))

In [346]:
url_df.to_csv('wikigrabber/all_people.csv')

### Explore Scrapy Results File

In [140]:
df = pd.read_csv('wikigrabber/all_people.csv', encoding='ISO-8859-1')
df

,wiki
0,'s_Gravesande
1,(Barbara)_Hazel_Guggenheim_King-Farlow_McKinley
2,1856_United_States_presidential_election#North...
3,1945ÃÂ¢ÃÂÃÂ1946_Massachusetts_legislature...
4,1947ÃÂ¢ÃÂÃÂ1948_Massachusetts_legislature...
...,...
71109,Ã Â½eljko_KomÃ Â¡iÃÂ
71110,Ã Â½eljko_KomÃ Â¡iÃÂ
71111,Ã Â½ivko_RadiÃ Â¡iÃÂ
71112,Ã Â½ivko_RadiÃ Â¡iÃÂ


In [ ]:
df['wiki'] = df['wiki'].apply(lambda x: x if not x.startswith())

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47952 entries, 0 to 47951
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   name       47251 non-null  object
 1   full_name  16425 non-null  object
 2   born       45753 non-null  object
 3   died       28590 non-null  object
dtypes: object(4)
memory usage: 1.5+ MB


In [20]:
# Delete blank records 
df_full = df.dropna(subset=['name'])
df_full

,name,full_name,born,died
0,Álvaro Santos Pereira,NaN,1972-01-07 00:00:00,NaN
1,Ángel Acebes Paniagua,Ángel Acebes Paniagua,1958-07-03 00:00:00,NaN
2,Álida España,Álida España,12 April 1924,8 April 1993 (aged 68)
3,Ángel Rozas,NaN,1950-03-22 00:00:00,NaN
4,Álvaro Nadal,NaN,1970-01-30 00:00:00,NaN
...,...,...,...,...
47947,Zury Ríos,Zury Mayté Ríos Sosa,1968-01-26 00:00:00,NaN
47948,Zsa Zsa Gabor,Sári Gábor,1917-02-06 00:00:00,"December 18, 2016"
47949,Zulima V. Farber,NaN,1944,NaN
47950,Zulfikar Ali Bhutto,NaN,1928-01-05 00:00:00,4 April 1979


In [21]:
df_full.to_csv('wikicrawler/people_no_blanks.csv')

In [14]:
# Drop all rows without a name
df_all_names = df_full.dropna(subset=['born', 'died'])
df_all_names                  

,name,full_name,born,died
2,Álida España,Álida España,12 April 1924,8 April 1993 (aged 68)
5,Árpád Göncz,NaN,1922-02-10 00:00:00,6 October 2015
12,Willem 's Gravesande,NaN,26 September 1688,28 February 1742
14,Åshild Hauan,NaN,1941-04-20 00:00:00,1 December 2017
17,Édouard Alphonse James de Rothschild,NaN,1868-02-24 00:00:00,30 June 1949
...,...,...,...,...
47937,Mehmet Ziya Gökalp,Muhammad Ziya,23 March 1876,25 October 1924
47938,"Zollie C. Steakley, Jr.",NaN,1908-08-29 00:00:00,"March 24, 1992"
47940,Zophar M. Mansur,NaN,1843-11-23 00:00:00,"March 28, 1914"
47948,Zsa Zsa Gabor,Sári Gábor,1917-02-06 00:00:00,"December 18, 2016"


In [18]:
df_all_names.to_csv('wikicrawler/people_full.csv')

In [459]:
df_all_names.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1233 entries, 0 to 1314
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   schools     835 non-null    object
 1   degrees     320 non-null    object
 2   name        1233 non-null   object
 3   full_name   581 non-null    object
 4   DOB         1163 non-null   object
 5   occupation  405 non-null    object
 6   spouses     697 non-null    object
 7   offspring   573 non-null    object
dtypes: object(8)
memory usage: 86.7+ KB


In [460]:
df_no_gaps = df_all_names.drop_duplicates()
df_no_gaps

,schools,degrees,name,full_name,DOB,occupation,spouses,offspring
0,London School of Economics,"MSc,BSc",George Soros,György Schwartz,1930-08-12 00:00:00,"Investor, hedge fund manager, author, philanth...",Susan Weber (historian),"Jonathan Soros,Alexander Soros"
1,"Yale University,University of Wyoming,Universi...","MA,BA",Dick Cheney,Richard Bruce Cheney,1941-01-30 00:00:00,NaN,Lynne Cheney,"Liz Cheney,Mary Cheney"
2,University of Fribourg,NaN,Klaus Schwab,NaN,1938-03-30 00:00:00,World Economic Forum,Hilde Schwab,2
3,"DePauw University,Indiana University, Indianap...","BA,JD",Dan Quayle,James Danforth Quayle,1947-02-04 00:00:00,NaN,Marilyn Quayle,Ben Quayle
4,"Dartmouth College,University College, Oxford,Y...","MA,BA,JD",Robert Reich,Robert Bernard Reich,1946-06-24 00:00:00,NaN,Clare Dalton,Sam Reich
...,...,...,...,...,...,...,...,...
1310,"Harvard University,BA,MBA",NaN,Theodore Roosevelt V,Theodore Roosevelt V,1942-11-27 00:00:00,NaN,Constance Lane Rogers,Theodore Roosevelt VI
1311,Harvard University,"BA,LLB",Charles Francis Adams III,NaN,1866-08-02 00:00:00,NaN,index.php?title=Frances Adams&action=edit&redl...,Charles Francis Adams IV
1312,"Harvard University,Yale University","JD,AB",Kermit Roosevelt III,NaN,1971-07-14 00:00:00,NaN,NaN,NaN
1313,"Yale University,Princeton University,Universit...","MA,BA,JD",Bob Taft,Robert Alphonso Taft III,1942-01-08 00:00:00,NaN,NaN,Anna Taft


In [461]:
df_no_gaps.to_csv('wikicrawler/people_data.csv')

In [462]:
df_degrees = df_no_gaps.dropna(subset=['schools', 'degrees'])

In [463]:
# Rearrange columns and sort on name
df_degrees = df_degrees[['name', 'full_name', 'DOB', 'occupation', 'schools', 'degrees', 'spouses', 'offspring']]
df_degrees.sort_values(by='name', ignore_index=True, inplace=True)
df_degrees

,name,full_name,DOB,occupation,schools,degrees,spouses,offspring
0,A. Clayton Spencer,Ava Clayton Spencer,1954-12-15 00:00:00,NaN,"Williams College,University of Oxford,Harvard ...","MA,BA,JD",Ash Carter,2
1,Abdul El-Sayed,Abdulrahman Mohamed El-Sayed,1984-10-31 00:00:00,NaN,"University of Michigan,Oriel College, Oxford,C...","MA,PhD,MD,BS",Sarah Jukaku,NaN
2,Alan Bersin,NaN,1946-10-15 00:00:00,NaN,"Harvard University,Yale University","BA,JD",Lisa Foster,NaN
3,Alan Chester Valentine,NaN,1901-02-23 00:00:00,NaN,"Swarthmore College,University of Pennsylvania,...","University of Pennsylvania,Balliol College, Ox...",Lucia Garrison Norton,Annie Laurie Buffinton
4,Alan Greenspan,NaN,1926-03-06 00:00:00,NaN,New York University,"PhD,MA,BA",Andrea Mitchell,NaN
...,...,...,...,...,...,...,...,...
308,William McAdoo,William Gibbs McAdoo Jr.,1863-10-31 00:00:00,NaN,"University of Tennessee, Knoxville",BA,Eleanor Wilson McAdoo,9
309,William McRae,William Allan McRae Jr.,1909-09-25 00:00:00,NaN,"University of Florida,University of Oxford,Fre...","B.A.,J.D.,B.Litt.,A.B.,B.A.,J.D.,B.Litt.,A.B.",NaN,NaN
310,William Taft,NaN,1945-09-13 00:00:00,NaN,"Yale University,Harvard University","BA,JD",Julia V. Taft,3
311,Wilson Elkins,Wilson Homer Elkins,1908-07-09 00:00:00,NaN,"University of Texas, Austin","MA,BA",Dorothy Blackburn,2


In [454]:
df_degrees.set_index('name', drop=True, inplace=True)
df_degrees

,full_name,DOB,occupation,schools,degrees,spouses,offspring
name,,,,,,,
A. Clayton Spencer,Ava Clayton Spencer,1954-12-15 00:00:00,NaN,"Williams College,University of Oxford,Harvard ...","MA,BA,JD",Ash Carter,2
Abdul El-Sayed,Abdulrahman Mohamed El-Sayed,1984-10-31 00:00:00,NaN,"University of Michigan,Oriel College, Oxford,C...","MA,PhD,MD,BS",Sarah Jukaku,NaN
Alan Bersin,NaN,1946-10-15 00:00:00,NaN,"Harvard University,Yale University","BA,JD",Lisa Foster,NaN
Alan Chester Valentine,NaN,1901-02-23 00:00:00,NaN,"Swarthmore College,University of Pennsylvania,...","University of Pennsylvania,Balliol College, Ox...",Lucia Garrison Norton,Annie Laurie Buffinton
Alan Greenspan,NaN,1926-03-06 00:00:00,NaN,New York University,"PhD,MA,BA",Andrea Mitchell,NaN
...,...,...,...,...,...,...,...
William McAdoo,William Gibbs McAdoo Jr.,1863-10-31 00:00:00,NaN,"University of Tennessee, Knoxville",BA,Eleanor Wilson McAdoo,9
William McRae,William Allan McRae Jr.,1909-09-25 00:00:00,NaN,"University of Florida,University of Oxford,Fre...","B.A.,J.D.,B.Litt.,A.B.,B.A.,J.D.,B.Litt.,A.B.",NaN,NaN
William Taft,NaN,1945-09-13 00:00:00,NaN,"Yale University,Harvard University","BA,JD",Julia V. Taft,3


In [455]:
# df_degrees = df_degrees.replace({np.nan:None})

In [456]:
def date_converter(x):
    if x:
        if '-' in x:
            updated = dt.strptime(str(x), '%Y-%m-%d %H:%M:%S')
        elif '/' in x:
            updated = dt.strptime(str(x), '%m/%d/%Y %H:%M:%S')
        else:
            return None
        print(updated)
        return updated.strftme('%Y-%m-%d')        
    else:
        return None

In [457]:
# df_degrees['DOB'] = np.vectorize(date_converter)(df_degrees['DOB'])
df_degrees['DOB'] = pd.to_datetime(df_degrees.DOB)
df_degrees['DOB'] = df_degrees['DOB'].dt.strftime('%Y-%m-%d')
df_degrees

ValueError: ('Unknown string format:', '1989 or 1990 (age\xa031–32)')

In [ ]:
df_degrees.to_csv('wikicrawler/people_data_full.csv')

## Rhodes Scholars

In [ ]:
rhodes_df = pd.read_csv('rhodescholars/rhodes_data.csv')
rhodes_df

In [ ]:
df_occupations = rhodes_df.merge(df_no_gaps, on='name', how='outer')
df_occupations

In [414]:
df_occupations.to_csv('wikicrawler/merged_occupations.csv')